# 18.369 pset 4

This is a template for the MPB calculations in pset 4, in the form of a Jupyter notebook.  You will need to install Meep and MPB via the Anaconda Python packages, which also allows you to run Jupyter notebooks in your browser.

In [ ]:
# do inline plots with Python's matplotlib library
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import math

# load the Meep and MPB modules in Python
import meep as mp
from meep import mpb

## Problem 2: Band gaps in MPB

The following code computes and plots the band diagram of a 1d-periodic structure, consisting of layers of two materials $\varepsilon_1 = 12$ and $\varepsilon_2 = 1$ with thicknesses $d_1$ and $d_2 = a - d_1$, respectively.

In 1d, both polarizations are equivalent, so we compute the "TM" ($E_z$ polarization) band structure only.  The following code computes it for $d_1 = 0.4a$, where we have a period $a=1$:

In [ ]:
eps1 = 12
eps2 = 1
d1 = 0.4
d2 = 1 - d1

ms = mpb.ModeSolver(                    
                    # unit cell is just size 1 in x direction (and zero size in y and z).
                    geometry_lattice=mp.Lattice(size=(1,0,0)),
                    
                    # we could define the unit cell by two objects (eps1 and eps2), but it
                    # is easier just to set the default material to eps2 and then add one
                    # object for eps1.
                    geometry=[mp.Block(center=(0,0,0), size=(d1,mp.inf,mp.inf),
                                       material=mp.Medium(epsilon=eps1))],
                    default_material=mp.Medium(epsilon=eps2),
    
                    # Let's set up the k points we want to compute.  Actually, to get the
                    # band gap we really only need k = pi/a, but we'll compute a range
                    # of k's in the irreducible Brillouin zone just so that we can make
                    # nice plots if we want.  Note that k is in units of 2pi/a, so, k=pi/a
                    # is just given as k=0.5.
                    k_points=mp.interpolate(9, [mp.Vector3(0), mp.Vector3(0.5)]),
                    
                    resolution=32, # this is plenty for the first few bands
                    num_bands=5 # the number of bands to compute (you may want more or less)
                    )

ms.run_tm() # TM and TE are equivalent in 1d, so we'll just do TM

Now, let's plot the computed band structure:

In [ ]:
freqs = ms.all_freqs # all the computed eigenfrequencies
kx = [k.x for k in ms.k_points] # get a list of the kx values

plt.figure(figsize=(9,6))
plt.plot(kx, freqs, "bo-")
plt.title("band structure for $d_1=%g$" % d1)
plt.xlabel("$k_x (2\pi/a)$")
plt.ylabel("frequency $(c/a)$")
plt.xlim(0,0.5)

Note that there is a whole sequence of gaps.  The first gap is between bands 1 and 2 at $k_x=0.5 (2\pi c/a)$.  We can compute the fractional gap via:

In [ ]:
# A note on indices in Python:
#      0 is the *first* index, so bands 1 and 2 are indices 0 and 1
#      -1 in Python is the *last* list element, so k=0.5 is index -1

gap_bottom = freqs[-1,0]  
gap_top =    freqs[-1,1]

# fractional gap = gap / midgap:
(gap_top - gap_bottom) / (0.5 * (gap_top + gap_bottom))

So, we have a 61.9% fractional gap.  You can also see this in the printed output of the `run_tm` function above — at the end of the output there are lines:
```
Gap from band 1 (0.1667885615162007) to band 2 (0.31635713735972837), 61.9144809491253%
Gap from band 2 (0.4237505813444828) to band 3 (0.5967638812678993), 33.90707457109922%
Gap from band 3 (0.733882991412184) to band 4 (0.7882439699552264), 7.142765343858948%
Gap from band 4 (0.9409357542580066) to band 5 (1.052560956250574), 11.19893515792054%
```

### part (a)

Now, compute the (fractional) gap size as a function of $d_1$.

To use a different $d_1$ with a given `ModeSolver` object like `ms` above, just evaluate:
```py
ms.geometry = [mp.Block(center=(0,0,0), size=(d1,mp.inf,mp.inf),
               material=mp.Medium(epsilon=eps1))]
```
for some different value of `d1`.   Then call `ms.run_tm()` to compute the bands, and compute the gap again as above.

(You might want to write a loop or something to do this for a bunch of values of `d1`.  The [`np.linspace` function](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linspace.html) might be helpful.)

**Note:** you can insert new "cells" for code etcetera by choosing *Insert Cell Below* from the *Insert* menu of the notebook, or using the keyboard shortcut "Ctrl-m b".

### part (b)

Plot the 1d TM band diagram for this structure, with $d_{1}$ given by the quarter wave thickness, showing the first five gaps. Also compute it for $d_{1}=0.12345$ (which I just chose randomly), and superimpose the two plots (plot the quarter-wave bands as solid lines and the other bands as dashed). What special features does the quarter-wave band diagram have?

## Problem 3: Defect modes in MPB

Here, you will create a ($E_z$/TM-polarized) "defect" mode by increasing the dielectric constant of a single layer by $\Delta\varepsilon$, pulling an eigenvalue down into the gap. The periodic structure will be the same as the one from the problem above, with the quarter-wave thickness $d_{1}=1/(1+\sqrt{12})$.

To model a defect in MPB, we have to use a "supercell" with N copies of the unit cell, where one of the copies has been changed.  This also means that we need to compute N times as many bands, because of the "band folding" (the reduction of the Brillouin zone).

For convenience, we'll define a function `supercell1(N, deps1)` Python function that sets up our `ModeSolver` object for a given N and $\Delta\varepsilon_1$:

In [ ]:
# define a function to create a supercell in the eps1 layer, with some default parameters:
def supercell1(N=15, deps1=0, eps1=12, eps2=1):
    d1 = math.sqrt(eps2) / (math.sqrt(eps1) + math.sqrt(eps2)) # quarter-wave thickness
    d2 = 1 - d1
    return mpb.ModeSolver(                    
        # unit cell a supercell: size N in x direction (and zero size in y and z).
        geometry_lattice=mp.Lattice(size=(N,0,0)),

        # To create the supercell geometry, we have to repeat the eps1 block N times,
        # and we can do this using the geometric_objects_lattice_duplicates function.
        # 
        # To create the defect, we'll simply append
        # a new object with eps1 + deps1 at the end of the geometry list
        # -- note that later objects take precedence over earlier objects,
        #    so by putting it at the end we ensure that the defect "overwrites"
        #    the whatever was previously there.
        geometry=mp.geometric_objects_lattice_duplicates(
                    mp.Lattice(size=(N,0,0)),
                    [mp.Block(center=(0,0,0), size=(d1,mp.inf,mp.inf),
                              material=mp.Medium(epsilon=eps1))]) +
                 [mp.Block(center=(0,0,0), size=(d1,mp.inf,mp.inf),
                           material=mp.Medium(epsilon=eps1+deps1))],
        
        default_material=mp.Medium(epsilon=eps2),

        # for computing a defect mode, k doesn't matter (if N is big enough),
        # so we'll just set k = 0
        k_points=[mp.Vector3(0)],

        resolution=32, # this is plenty for the first few bands
        
        # because of the folding, the first band (before the gap) will be folded
        # N times.  So, we need to compute N bands plus some extra bands to
        # get whatever defect states lie in the gap.  We'll just use 1 extra band,
        # but you'll need to increase this to see higher-order defect modes.
        num_bands=N+1
        )

Before we do anything, it is *always* a good idea to plot the $\varepsilon(x)$ function returned by `get_epsilon()` in order to make sure it is what we think it is.   Here, we expect to get N copies of the unit cell, with $\varepsilon_1 = 12 - \Delta\varepsilon_1 = 8$ at $x=0$:

In [ ]:
sc = supercell1(N=15, deps1=-4)
sc.run_tm()

In [ ]:
N = 15
x = np.linspace(-N/2, N/2, N*sc.resolution[0])
plt.plot(x, sc.get_epsilon())
plt.title("supercell $\epsilon$, with defect")

Looks good!

## part (a)

When there is no defect ($\Delta\varepsilon = 0$), plot out the band diagram $\omega(k)$ for the $N=5$ supercell, and show that it corresponds to the band diagram of problem 2 “folded” as expected.

You'll want to change `N` to `5`, and also change the k points to an array:

In [ ]:
sc = supercell1(N=5, deps1=0)

# Compute the band structure for an array of k points.  Note that the k points
# are in units of the reciprical lattice, so kx=0.5 is still the edge of the 
# Brillouin zone (but is not the same kx=pi/a!).
sc.k_points = mp.interpolate(19, [mp.Vector3(0), mp.Vector3(0.5)])

# compute some more bands so that we get a clear picture of what is going on
sc.num_bands = 20

sc.run_tm()

In [ ]:
kx = [k.x for k in sc.k_points]
freqs = sc.all_freqs

# plot freqs vs kx ...

### part (b)

Create a defect mode (a mode that lies in the band gap of the periodic structure) by increasing the $\varepsilon$ of a single $\varepsilon_{1}$ layer by $\Delta\varepsilon=1$, and plot the $E_{z}$ field pattern. Do the same thing by increasing a single $\varepsilon_{2}$ layer. Which mode is even/odd around the mirror plane of the defect? Why?

For the $\Delta\varepsilon_1$ layer, we can use our `supercell1` function from above:

In [ ]:
N = 100

sc = supercell1(N=N, deps1=1)
sc.run_tm(mpb.fix_efield_phase)

In [ ]:
# get the Ez field of band N+1, which should be the mode in the gap for deps1 > 0
ez = sc.get_efield(N+1)[:,0,0,2]

x = np.linspace(-N/2, N/2, N*sc.resolution[0])

# for fun, we'll plot on both a linear and a log scale
plt.figure(figsize=(13,8))
plt.subplot(1,2,1)
plt.plot(x, np.real(ez))
plt.title("supercell $\Re[E_z]$ for $\Delta\epsilon_1 = 1$")
plt.xlabel("$x (a)$")
plt.subplot(1,2,2)
plt.semilogy(x, np.real(ez))
plt.title("supercell $|E_z|$ for $\Delta\epsilon_1 = 1$")
plt.xlabel("$x (a)$")

### part (b) continued

Now, to do the same thing but for $\Delta\varepsilon_2 = 1$, you'll have to modify the `supercell1` function (maybe defining an analogous `supercell2` function) to create a defect in a $\varepsilon_2$ layer.


In [ ]:
# insert your code here

### part (c)

Gradually increase the $\varepsilon$ of a single $\varepsilon_{2}$ layer, and plot the defect \omega as a function of $\Delta\varepsilon_2$ as the frequency sweeps across the gap. At what $\Delta\varepsilon_2$ do you get two defect modes in the gap? Plot the $E_{z}$ of the second defect mode.

Be careful to increase the size of the supercell for modes near the edge of the gap, which are only weakly localized.  You may need to increase `num_bands` in order to see a second mode in the gap (if any).

In [ ]:
# insert your code here

### part (d)

The mode must decay exponentially far from the defect (multiplied by an $e^{i\frac{\pi}{a}x}$ sign oscillation and the periodic Bloch envelope, of course). From the $E_{z}$ field computed by MPB, extract this asympotic exponential decay rate (i.e. $\kappa$ if the field decays $\sim e^{-\kappa x}$) and plot this rate as a function of $\omega$, for the first defect mode, as you increase $\varepsilon_{2}$ as above (vary $\varepsilon_{2}$ so that $\omega$ goes from the top of the gap to the bottom).

You'll probably want to increase the supercell size to get a clear picture of the decay rate, especially for modes near the edge of the gap.

Note that a good estimate for $\kappa$ is simply $$\kappa \approx \log|E_z(x)/E_z(x+a)|$$
for some suitable $x \gg 0$ (maybe halfway between the defect and the edge of the computational cell.   From a Python `ez` array, you can compute `kappa = math.log(np.abs(ez[x*resolution]/ez[(x+1)*resolution]))` for a suitable choice of `x`.  Alternatively, you could do some form of curve-fitting.

In [ ]:
# insert your code here